In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 1
%aimport numpy_api_stats

In [3]:
import os
os.environ['GOOGLE_CRED_DIR']=os.path.join(os.environ['HOME'],'creds')
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=os.path.join(os.environ['GOOGLE_CRED_DIR'],'apt-footing-235018-aeb185ac9e31.json')

#### Investigate bigQuery public github dataset contents using bqhelper

In [16]:
#from google.cloud import bigquery
github_repos = ns.BigQueryHelper(active_project= "bigquery-public-data", 
                                       dataset_name = "github_repos")

In [17]:
github_repos.list_tables()

['commits',
 'contents',
 'files',
 'languages',
 'licenses',
 'sample_commits',
 'sample_contents',
 'sample_files',
 'sample_repos']

In [18]:
api_results = ns.BigQueryHelper(active_project= "apt-footing-235018", 
                                       dataset_name = "NumpyAPI")
api_results.list_tables()

['content',
 'files',
 'functions',
 'imports',
 'methods',
 'projects',
 'results',
 'results2',
 'sample_results',
 'watched',
 'watched_python',
 'watched_sample_files']

In [19]:
github_repos.table_schema('files')

,name,type,mode,description
0,repo_name,STRING,NULLABLE,None
1,ref,STRING,NULLABLE,None
2,path,STRING,NULLABLE,None
3,mode,INTEGER,NULLABLE,None
4,id,STRING,NULLABLE,None
5,symlink_target,STRING,NULLABLE,None


### Construct API datasets

In [5]:
import numpy as np
import numpy_api_stats as ns

api = [(x, type(np.__getattribute__(x))) for x in dir(np) if not x.startswith('__')]

#### Query Constructed for API functions..  

In [6]:
api[1][1].__name__

'type'

In [7]:
funs = ns.select(api,'function')
mods = ns.select(api,'module')
ints = ns.select(api,'int')
floats = ns.select(api,'float')

In [9]:
full_api = [(x[0],type( np.__getattribute__(x[0])).__name__) for x in api ]

In [10]:
# Use all to build full query
api_list=ns.build_api_list(full_api)

#Just a few for Demonstration
api_list = funs[:3]+mods[:3]+ints[:3]+floats[:3]

In [12]:
import numpy_api_stats as ns
apq = ns.API_QUERY_FACTORY(api_list)
print(apq.query)

SELECT
REGEXP_MATCH(c.content,'(import\s+numpy\.|from numpy\._distributor_init\s+import\s+[a-zA-Z0-9_]+)' ) AS numpy_module__distributor_init,
REGEXP_MATCH(c.content,'(import\s+numpy\.|from numpy\._globals\s+import\s+[a-zA-Z0-9_]+)' ) AS numpy_module__globals,
REGEXP_MATCH(c.content,'(import\s+numpy\.|from numpy\._import_tools\s+import\s+[a-zA-Z0-9_]+)' ) AS numpy_module__import_tools,
REGEXP_MATCH( c.content,'(np\.|numpy\.)add_newdoc\(\s?[A-Za-z0-9_.\(\)]*\s?\)' ) AS numpy_function_add_newdoc,
REGEXP_MATCH( c.content,'(np\.|numpy\.)alen\(\s?[A-Za-z0-9_.\(\)]*\s?\)' ) AS numpy_function_alen,
REGEXP_MATCH( c.content,'(np\.|numpy\.)all\(\s?[A-Za-z0-9_.\(\)]*\s?\)' ) AS numpy_function_all
FROM (SELECT
*
FROM
[bigquery-public-data:github_repos.contents] AS c
INNER JOIN
(SELECT
*
FROM
[bigquery-public-data:github_repos.files] AS f
WHERE
(
f.path LIKE '%.py' 
OR
f.path LIKE '%.ipynb'
) ) AS p
ON
c.id = p.id)


### AFter copying and pasting the above query into BigQuery and saving as sample_results... 

#####Unfortunately this doesn't work https://github.com/dask/dask/issues/3121 so we manually export results.

In [ ]:
#Pull data from bigQuery
#r = api_results.query_to_pandas_safe('SELECT * FROM `apt-footing-235018.NumpyAPI.sample_results`')
#r